<a href="https://colab.research.google.com/github/Abhish777/Image-Augmentation-and-Segmentation/blob/master/Image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
originalImage = cv2.imread("/content/holi.png")
originalImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)

Transforming the image

In [ ]:
reshapedImage = np.float32(originalImage.reshape(-1, 3))


Defining the number of cluster we want to find

In [ ]:
numberOfClusters = 2

stopCriteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.1)

kmeans clustering algorithm. This will return us a list with the centroids and a list with all the pixels

In [ ]:
ret, labels, clusters = cv2.kmeans(reshapedImage, numberOfClusters, None, stopCriteria, 10, cv2.KMEANS_RANDOM_CENTERS)

clusters = np.uint8(clusters)

intermediateImage = clusters[labels.flatten()]
clusteredImage = intermediateImage.reshape((originalImage.shape))

cv2.imwrite("clusteredImage.jpg", clusteredImage)

True

# Canny edge detection in opencv


In [ ]:
# Remove 1 cluster from image and apply canny edge detection
removedCluster = 1

cannyImage = np.copy(originalImage).reshape((-1, 3))
cannyImage[labels.flatten() == removedCluster] = [0, 0, 0]

cannyImage = cv2.Canny(cannyImage,100,200).reshape(originalImage.shape)
cv2.imwrite("cannyImage.jpg", cannyImage)

True

# Finding contours using opencv


In [ ]:
initialContoursImage = np.copy(cannyImage)
imgray = cv2.cvtColor(initialContoursImage, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(imgray, 50, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(initialContoursImage, contours, -1, (0,0,255), cv2.CHAIN_APPROX_SIMPLE)
cv2.imwrite("initialContoursImage.jpg", initialContoursImage)

True

In [ ]:
cnt = contours[0]
largest_area=0
index = 0
for contour in contours:
    if index > 0:
        area = cv2.contourArea(contour)
        if (area>largest_area):
            largest_area=area
            cnt = contours[index]
    index = index + 1

biggestContourImage = np.copy(originalImage)
cv2.drawContours(biggestContourImage, [cnt], -1, (0,0,255), 3)
cv2.imwrite("biggestContourImage.jpg", biggestContourImage)

True

# Image quantification with k-means

Loading an Image

In [ ]:
from PIL import Image
I = Image.open("holi.png")


Transforming into numpy array

In [ ]:
a = np.asarray(I,dtype=np.float32)/255
plt.figure(figsize=(12,12))
plt.imshow(a)
plt.axis('off')
plt.show()

Counting number of pixels and colors

In [ ]:
w, h = I.size
colors = I.getcolors(w * h)
num_colors = len(colors) 
num_pixels = w*h 

print ('Number of pixels = ', num_pixels)
print ('Number of colors = ', num_colors)

Number of pixels =  291100
Number of colors =  118709


Re arranging the matrix to apply K-means

In [ ]:
x, y, z = a.shape
print('a shape ', a.shape)

a1 = a.reshape(x*y, z)
print('a1 shape ', a1.shape)

a shape  (410, 710, 4)
a1 shape  (291100, 4)


Grouping the colors into 60 groups

In [ ]:
n = 60
k_means = KMeans(n_clusters=n)
k_means.fit(a1)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=60, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

The final centroids are the new colors and each pixes has been assigned a label

In [ ]:
centroids = k_means.cluster_centers_
labels = k_means.labels_

Reconstructing the matrix of the image from the labels and the colors

In [ ]:
print('centroids shape ', centroids.shape)
print('labels shape ', labels.shape)

centroids shape  (60, 4)
labels shape  (291100,)


In [ ]:
a2 = centroids[labels]
print('a2 shape ', a2.shape)

a3 = a2.reshape(x,y,z)
print('a3 shape ', a3.shape)

a2 shape  (291100, 4)
a3 shape  (410, 710, 4)


Viewing the image with only 60 colors

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(a3)
plt.axis('off')
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [ ]:
a4 = np.floor(a3*255)
a5 = a4.astype(np.uint8)

I1 = Image.fromarray(a5)
I1.save("colors.png")

In [ ]:
w, h = I1.size
colors = I1.getcolors(w * h)
num_colors = len(colors) 
num_pixels = w*h 

print ('Number of pixels = ', num_pixels)
print ('Number of colors = ', num_colors)

Number of pixels =  291100
Number of colors =  60
